In [1]:
import numpy as np
import pandas as pd
from importlib import reload

In [2]:
# define constants
TRACE_FILE_NAME = 'data/Skype_HongKong.pcapng' # replace with your raw trace .pcapng/.pcap file name
TRACE_TCP_PACKET_FEATURE_FILE_NAME = 'data/Skype_HongKong_tcp_pkt.csv' # replace with your favorite tcp packet feature .csv file name
TRACE_UDP_PACKET_FEATURE_FILE_NAME = 'data/Skype_HongKong_udp_pkt.csv' # replace with your favorite udp packet feature .csv file name
TRACE_PACKET_FEATURE_FILE_NAME = 'data/Skype_HongKong_pkt.csv' # replace with your favorite packet feature .csv file name
LOCAL_IP = '172.16.26.207' # your local ip

## Extract TCP packets

In [3]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.tcp_generate(TRACE_FILE_NAME,TRACE_TCP_PACKET_FEATURE_FILE_NAME)

Packet feature file already exists.
CPU times: user 151 µs, sys: 93 µs, total: 244 µs
Wall time: 223 µs


In [40]:
# read in packet feature csv file and do some transformation
import ipaddress
tcp_pkt_feature_df = pd.read_csv(TRACE_TCP_PACKET_FEATURE_FILE_NAME)
filterer = tcp_pkt_feature_df.apply(lambda row:(not pd.isnull(row['ip.src']) and ipaddress.IPv4Address(row['ip.src']).is_global) or (not pd.isnull(row['ip.dst']) and ipaddress.IPv4Address(row['ip.dst']).is_global),axis=1)
tcp_pkt_feature_df = tcp_pkt_feature_df[filterer]
record_num = tcp_pkt_feature_df.shape[0]
tcp_pkt_feature_df['remote_ip'] = tcp_pkt_feature_df.apply(lambda row:row['ip.dst'] if ipaddress.IPv4Address(row['ip.dst']).is_global else row['ip.src'],axis=1) if record_num > 0 else None
tcp_pkt_feature_df['remote_ip2num'] = tcp_pkt_feature_df.apply(lambda row:int(ipaddress.IPv4Address(row['remote_ip'])),axis=1) if record_num > 0 else None
tcp_pkt_feature_df['protocol'] = 'tcp' if record_num > 0 else None
tcp_pkt_feature_df['is_tcp'] = 1 if record_num > 0 else None
tcp_pkt_feature_df['is_udp'] = 0 if record_num > 0 else None
tcp_pkt_feature_df.rename(columns={'tcp.len':'pkt_len'},inplace=True)

In [41]:
# view the shape of the dataset: (number of records, number of features)
tcp_pkt_feature_df.shape

(3678, 17)

In [42]:
# view the data types for each feature
tcp_pkt_feature_df.dtypes

ip.src                  object
ip.dst                  object
tcp.srcport              int64
tcp.dstport              int64
pkt_len                  int64
frame.time_relative    float64
tcp.seq                  int64
tcp.ack                  int64
tcp.flags.ack            int64
tcp.flags.syn            int64
tcp.flags.fin            int64
tcp.stream               int64
remote_ip               object
remote_ip2num            int64
protocol                object
is_tcp                   int64
is_udp                   int64
dtype: object

In [43]:
# view the statistical features of each numerical feature
tcp_pkt_feature_df.describe()

,tcp.srcport,tcp.dstport,pkt_len,frame.time_relative,tcp.seq,tcp.ack,tcp.flags.ack,tcp.flags.syn,tcp.flags.fin,tcp.stream,remote_ip2num,is_tcp,is_udp
count,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3.678000e+03,3678.0,3678.0
mean,29043.695487,25612.809951,265.895867,252.438116,7407.414899,7616.953235,0.961120,0.053290,0.050843,232.175095,1.239089e+09,1.0,0.0
std,24214.322178,24538.058716,469.979511,243.678426,15084.979280,15551.347280,0.193335,0.224641,0.219707,243.597741,9.101231e+08,0.0,0.0
min,80.000000,80.000000,0.000000,2.271637,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.240264e+08,1.0,0.0
25%,443.000000,443.000000,0.000000,16.402931,208.000000,235.000000,1.000000,0.000000,0.000000,38.000000,3.015624e+08,1.0,0.0
50%,49491.000000,40033.000000,0.000000,130.084942,1848.000000,2395.000000,1.000000,0.000000,0.000000,117.000000,8.830733e+08,1.0,0.0
75%,49590.000000,49571.000000,311.000000,529.345451,5549.750000,5657.000000,1.000000,0.000000,0.000000,512.000000,1.752893e+09,1.0,0.0
max,63232.000000,63232.000000,1460.000000,660.412777,93784.000000,94093.000000,1.000000,1.000000,1.000000,693.000000,3.427780e+09,1.0,0.0


In [44]:
# view the first 5 records
tcp_pkt_feature_df.head()

,ip.src,ip.dst,tcp.srcport,tcp.dstport,pkt_len,frame.time_relative,tcp.seq,tcp.ack,tcp.flags.ack,tcp.flags.syn,tcp.flags.fin,tcp.stream,remote_ip,remote_ip2num,protocol,is_tcp,is_udp
24,90.217.83.107,192.168.1.163,62530,6769,0,2.271637,1,1,1,0,1,2,90.217.83.107,1524192107,tcp,1,0
25,192.168.1.163,90.217.83.107,6769,62530,0,2.271725,1,2,1,0,0,2,90.217.83.107,1524192107,tcp,1,0
26,192.168.1.163,90.217.83.107,6769,62530,0,2.271783,1,2,1,0,1,2,90.217.83.107,1524192107,tcp,1,0
27,90.217.83.107,192.168.1.163,62530,6769,0,2.381490,2,2,1,0,0,2,90.217.83.107,1524192107,tcp,1,0
64,52.162.161.12,192.168.1.163,443,49445,1448,3.253662,1,1,1,0,0,5,52.162.161.12,883073292,tcp,1,0


## Extract UDP packets

In [12]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(TRACE_FILE_NAME,TRACE_UDP_PACKET_FEATURE_FILE_NAME,True)

Conversion done


No error


CPU times: user 3.91 ms, sys: 5.44 ms, total: 9.36 ms
Wall time: 28.4 s


In [13]:
udp_pkt_feature_df = pd.read_csv(TRACE_UDP_PACKET_FEATURE_FILE_NAME)
udp_pkt_feature_df

,ip.src,ip.dst,udp.srcport,udp.dstport,udp.length,frame.time_relative,udp.stream
0,192.168.1.1,239.255.255.250,52257,1900,183,0.531698,0
1,192.168.1.163,192.168.1.1,55446,53,43,3.255696,1
2,192.168.1.163,192.168.1.1,57465,53,55,3.275782,2
3,192.168.1.1,192.168.1.163,53,57465,104,3.289003,2
4,192.168.1.1,192.168.1.163,53,55446,149,3.335964,1
5,192.168.1.163,192.168.1.1,51290,53,59,3.343116,3
6,192.168.1.1,192.168.1.163,53,51290,59,3.445033,3
7,192.168.1.163,90.217.83.107,6769,28331,26,6.203742,4
8,90.217.83.107,192.168.1.163,28331,6769,34,6.310376,4
9,"192.168.1.163,90.217.83.107","90.217.83.107,192.168.1.163",28331,6769,34,6.310441,4


In [65]:
# read in packet feature csv file and do some transformation
import ipaddress
def filter_illegal(row):
    try:
        ipaddress.IPv4Address(row['ip.src'])
        ipaddress.IPv4Address(row['ip.dst'])
        return (not pd.isnull(row['ip.src']) and ipaddress.IPv4Address(row['ip.src']).is_global) or (not pd.isnull(row['ip.dst']) and ipaddress.IPv4Address(row['ip.dst']).is_global)
    except ValueError as e:
        print(e)
        return False                                                                                
                                                                                                
udp_pkt_feature_df = pd.read_csv(TRACE_UDP_PACKET_FEATURE_FILE_NAME)
filterer = udp_pkt_feature_df.apply(filter_illegal,axis=1)
udp_pkt_feature_df = udp_pkt_feature_df[filterer]
record_num = udp_pkt_feature_df.shape[0]
udp_pkt_feature_df['remote_ip'] = udp_pkt_feature_df.apply(lambda row:row['ip.dst'] if ipaddress.IPv4Address(row['ip.dst']).is_global else row['ip.src'],axis=1) if record_num > 0 else None
udp_pkt_feature_df['remote_ip2num'] = udp_pkt_feature_df.apply(lambda row:int(ipaddress.IPv4Address(row['remote_ip'])),axis=1) if record_num > 0 else None
udp_pkt_feature_df['protocol'] = 'udp' if record_num > 0 else None
udp_pkt_feature_df['is_tcp'] = 0 if record_num > 0 else None
udp_pkt_feature_df['is_udp'] = 1 if record_num > 0 else None
udp_pkt_feature_df.rename(columns={'udp.length':'pkt_len'},inplace=True)

Expected 4 octets in '192.168.1.163,90.217.83.107'
Expected 4 octets in '192.168.1.1,192.168.1.163'
Expected 4 octets in '192.168.1.1,192.168.1.163'
Expected 4 octets in '192.168.1.1,192.168.1.163'
Expected 4 octets in '192.168.1.1,192.168.1.163'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'na

In [66]:
# view the shape of the dataset: (number of records, number of features)
udp_pkt_feature_df.shape

(365042, 12)

In [67]:
# view the data types for each feature
udp_pkt_feature_df.dtypes

ip.src                  object
ip.dst                  object
udp.srcport              int64
udp.dstport              int64
pkt_len                  int64
frame.time_relative    float64
udp.stream               int64
remote_ip               object
remote_ip2num            int64
protocol                object
is_tcp                   int64
is_udp                   int64
dtype: object

In [68]:
# view the statistical features of each numerical feature
udp_pkt_feature_df.describe()

,udp.srcport,udp.dstport,pkt_len,frame.time_relative,udp.stream,remote_ip2num,is_tcp,is_udp
count,365042.000000,365042.000000,365042.000000,365042.000000,365042.000000,3.650420e+05,365042.0,365042.0
mean,14406.164847,18850.348343,760.436632,373.753011,93.012634,1.750116e+09,0.0,1.0
std,12992.739403,12960.373591,369.584589,166.696518,2.767278,7.340355e+07,0.0,0.0
min,123.000000,80.000000,11.000000,0.531698,0.000000,2.779690e+07,0.0,1.0
25%,3480.000000,3480.000000,551.000000,229.997368,93.000000,1.747766e+09,0.0,1.0
50%,3480.000000,29750.000000,889.000000,373.524868,93.000000,1.747766e+09,0.0,1.0
75%,29750.000000,29750.000000,1057.000000,517.885260,93.000000,1.747766e+09,0.0,1.0
max,64888.000000,62902.000000,1321.000000,663.106770,211.000000,4.026532e+09,0.0,1.0


In [69]:
# view the first 5 records
udp_pkt_feature_df.head()

,ip.src,ip.dst,udp.srcport,udp.dstport,pkt_len,frame.time_relative,udp.stream,remote_ip,remote_ip2num,protocol,is_tcp,is_udp
0,192.168.1.1,239.255.255.250,52257,1900,183,0.531698,0,239.255.255.250,4026531834,udp,0,1
7,192.168.1.163,90.217.83.107,6769,28331,26,6.203742,4,90.217.83.107,1524192107,udp,0,1
8,90.217.83.107,192.168.1.163,28331,6769,34,6.310376,4,90.217.83.107,1524192107,udp,0,1
26,192.168.1.163,239.255.255.250,51602,1900,141,8.155950,13,239.255.255.250,4026531834,udp,0,1
27,192.168.1.163,239.255.255.250,51602,1900,140,8.155951,13,239.255.255.250,4026531834,udp,0,1


## Combine TCP with UDP packets

In [70]:
# combine dataframes
pkt_feature_df = tcp_pkt_feature_df[['remote_ip2num','is_tcp','is_udp','pkt_len']].append(udp_pkt_feature_df[['remote_ip2num','is_tcp','is_udp','pkt_len']],ignore_index=True)

In [71]:
# shape
pkt_feature_df.shape

(368720, 4)

In [72]:
###### column types
pkt_feature_df.dtypes

remote_ip2num    int64
is_tcp           int64
is_udp           int64
pkt_len          int64
dtype: object

In [73]:
# describe
pkt_feature_df.describe()

,remote_ip2num,is_tcp,is_udp,pkt_len
count,3.687200e+05,368720.000000,368720.000000,368720.000000
mean,1.745019e+09,0.009975,0.990025,755.503564
std,1.271758e+08,0.099376,0.099376,373.962863
min,2.779690e+07,0.000000,0.000000,0.000000
25%,1.747766e+09,0.000000,1.000000,526.000000
50%,1.747766e+09,0.000000,1.000000,885.000000
75%,1.747766e+09,0.000000,1.000000,1056.000000
max,4.026532e+09,1.000000,1.000000,1460.000000


In [74]:
# head 5 records
pkt_feature_df.head()

,remote_ip2num,is_tcp,is_udp,pkt_len
0,1524192107,1,0,0
1,1524192107,1,0,0
2,1524192107,1,0,0
3,1524192107,1,0,0
4,883073292,1,0,1448


In [75]:
# write to csv
pkt_feature_df.to_csv(TRACE_PACKET_FEATURE_FILE_NAME, index=False)